In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
import random 
import copy

from sklearn import datasets 
from sklearn.neural_network import MLPClassifier 
from sklearn.metrics import accuracy_score 
from sklearn.model_selection import train_test_split 
from random import randint 
from sklearn.metrics import mean_absolute_error as mae 
from tqdm import tqdm 

In [2]:
mnist = datasets.load_digits()

X = mnist.data 
y = mnist.target 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [3]:
print(X_train.shape)
print(y_train.shape)

(1257, 64)
(1257,)


In [4]:
# initialize population 
def intialize_population(pop_size):
    pop = [[]] * pop_size   # create a list(=A) of lists(=B). There are 'pop_size' lists(=B) in the list(=A)
    activation = ['identity', 'logistic', 'tanh', 'relu']
    solver = ['sgd', 'adam']
    pop = [[random.choice(activation), random.choice(solver), randint(2, 4), randint(2, 4)] for i in tqdm(range(0, pop_size))]
    return pop

In [5]:
def crossover_mlp(parent1, parent2):
    child = [parent1[0], parent2[1], parent1[2], parent2[3]]
    return child 

In [6]:
def mutation_mlp(children, prob):
    for c in range(0, len(children)):
        if np.random.rand() > prob:
            k = randint(2, 3)
            children[c][k] = int(children[c][k]) + randint(-3, 6)

    children = np.array(children)
    return children 

In [7]:
def fitness_mlp(pop, X_train, y_train, X_test, y_test):
    fitness = []

    for individual in tqdm(pop):
        clf = MLPClassifier(learning_rate_init=0.1, 
                            activation=individual[0],
                            solver = individual[1], 
                            hidden_layer_sizes=(int(individual[2]), int(individual[3])),  
                            max_iter=2000)
        
        try:
            clf.fit(X_train, y_train)
            f = accuracy_score(clf.predict(X_test), y_test)
            fitness.append([f, clf, individual])
        except:
            print('--- WARNING --- INVALID VALUE ENCOUNTERED ---')

    fitness = np.array(fitness)
    return fitness 

In [8]:
def ga_main_mlp(X_train, y_train, X_test, y_test, generations=10, pop_size=20, mutation_prob=0.1):
    # generate population 
    print("Generating initial population ...")
    population = intialize_population(pop_size) 

    for gen in range(generations):
        print(f"\n --- Generation: {gen} | Population: {len(population)}")

        # a new individual will go to this 'new_population'
        new_population = list()

        print("Calculating fitness for each individual ...")
        fitness = fitness_mlp(population, X_train, y_train, X_test, y_test)

        # sort all individuals by fitness(=accuracy) in decreasing order
        fitness_sorted = np.array(list(reversed(sorted(fitness, key=lambda x: x[0]))))

        num_individuals = len(fitness_sorted)

        # 80 percent of them remain the same 
        num_remainings = int(num_individuals * 0.85)
        for idx in range(num_remainings):
            new_population.append(fitness_sorted[:, -1][idx])

        # 20 percent of them perform crossover and mutation 
        num_cross_mutate = int(num_individuals * 0.15)

        # flatten and shuffle fitness_sorted
        individuals = fitness_sorted[:, 2].flatten()
        np.random.shuffle(individuals)

        # select half of the individuals for each parent
        half = num_individuals // 2
        parent_1 = individuals[:half]
        parent_2 = individuals[half:half*2]

        child_1 = [crossover_mlp(parent_1[i], parent_2[i]) for i in range(num_cross_mutate)]
        child_2 = [crossover_mlp(parent_2[i], parent_1[i]) for i in range(num_cross_mutate)]

        child_1 = mutation_mlp(child_1, mutation_prob)
        child_2 = mutation_mlp(child_2, mutation_prob)

        for idx2 in range(len(child_1)):
            new_population.append(child_1[idx2])
        
        for idx3 in range(len(child_2)):
            new_population.append(child_2[idx3])

        new_population = np.array(new_population)
        
        population = copy.deepcopy(new_population)

        print(f"Best Fitness: {fitness_sorted[0]}")

In [9]:
ga_main_mlp(X_train, y_train, X_test, y_test, generations=10, pop_size=10, mutation_prob=0.1)

Generating initial population ...


100%|██████████| 10/10 [00:00<00:00, 52958.38it/s]



 --- Generation: 0 | Population: 10
Calculating fitness for each individual ...


100%|██████████| 10/10 [00:03<00:00,  2.53it/s]
<ipython-input-7-2657c7e4f9ea>:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  fitness = np.array(fitness)


Best Fitness: [0.8592592592592593
 MLPClassifier(activation='identity', hidden_layer_sizes=(3, 4),
               learning_rate_init=0.1, max_iter=2000)
 list(['identity', 'adam', 3, 4])]

 --- Generation: 1 | Population: 10
Calculating fitness for each individual ...


...


<ipython-input-7-2657c7e4f9ea>:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  fitness = np.array(fitness)




 --- Generation: 9 | Population: 8
Calculating fitness for each individual ...


100%|██████████| 8/8 [00:02<00:00,  2.88it/s]

Best Fitness: [0.9092592592592592
 MLPClassifier(activation='logistic', hidden_layer_sizes=(10, 8),
               learning_rate_init=0.1, max_iter=2000, solver='sgd')
 array(['logistic', 'sgd', '10', '8'], dtype='<U21')]



<ipython-input-7-2657c7e4f9ea>:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  fitness = np.array(fitness)
